# PointNet for particle flow

<div class="alert alert-block alert-success">

Weights & Biases **sweeps** provide a powerful way to automate hyperparameter optimization while maintaining comprehensive experiment tracking. With sweeps, you can systematically explore different hyperparameter configurations using various search strategies.

### Key Features
- Automated scheduling of training runs with different hyperparameters
- Support for multiple search strategies:
  - Grid search: exhaustive search over specified parameter values
  - Random search: sampling from defined distributions
  - Bayesian optimization: adaptive sampling based on previous results
- hyperparam space can be defined as set of values or sampling distribution
- Rich visualization and comparison tools in the W&B UI

### Example: Setting up a Sweep

```python
import wandb

# Define sweep configuration
sweep_config = {
    'method': 'random',  # or 'grid', 'bayes'
    'metric': {'name': 'validation_loss','goal': 'minimize'},
    'parameters': {
        'learning_rate': { 'min': 1e-5, 'max': 1e-2,
                          'distribution': 'log_uniform' },
        'batch_size': { 'values': [16, 32, 64] },
        'optimizer': { 'values': ['adam', 'sgd'] }
    }
}

# Initialize sweep
sweep_id = wandb.sweep(sweep_config, project="sweep_demo")

# Define training function
def train():
    run = wandb.init()
    
    # Access hyperparameters from sweep
    config = wandb.config
    
    # Your training code here
    # ...
    
    # Log metrics
    wandb.log({"validation_loss": val_loss})

# Start sweep agent
wandb.agent(sweep_id, train, count=5)  # Run 5 experiments
```

This configuration will automatically run multiple experiments with different combinations of learning rates, batch sizes, and optimizers, tracking all results in the W&B dashboard.


**Main changes:** 

- repeat full pipeline for different learning rates exploiting wandb sweeps

</div>

## Problem

This dataset contains a Monte Carlo simulation of $\rho^{\pm} \rightarrow \pi^{\pm} + \pi^0$ decays and the corresponding detector response. Specifically, the data report the measured response of **i) tracker** and **ii) calorimeter**, along with the true pyshical quantitites that generated those measurements.

<div class="alert alert-block alert-info">
This means that we expect one track per event, with mainly two energy blobs (clusters of cells) in the calorimeter.
</div>

The final **goal** is to associate the cell signals observed in the calorimeter to the track that caused those energy deposits.

## Method

The idea is to leverage a **point cloud** data representation to combine tracker and calorimeter information so to associate cell hits to the corresponding track. We will use a [**PointNet**](https://openaccess.thecvf.com/content_cvpr_2017/papers/Qi_PointNet_Deep_Learning_CVPR_2017_paper.pdf) model that is capable of handling this type of data, framed as a **semantic segmentation** approach. More precisely, this means that:
- we represent each hit in the detector as a point in the point cloud: x, y, z coordinates + additional features ("3+"-dimensional point)
- the **learning task** will be binary classification at hit level: for each cell the model learns whether its energy comes mostly from the track (class 1) or not (class 0)

## Data structure

<div class="alert alert-block alert-info">

This dataset is organized as follows:
 - for each event, we create a **sample** (i.e. point cloud)
 - each sample contains all hits in a cone around a track of the event, called **focal track**
     - the cone includes all hits within some $\Delta R$ distance of the track
     - if an event has multiple tracks, then we have more samples per event
     - since different samples have possibly different number of hits, **we pad all point clouds to ensure they have same size** (needed since the model requires inputs of same size)

</div>

## Settings & config

This section collects all configuration variables and training/model hyperparameters. 

The idea is to put it at the top so that it is easy to find and edit.

In [1]:
import sys
import numpy as np
import pandas as pd
from pathlib import Path

import matplotlib.pyplot as plt

# path settings
REPO_BASEPATH = Path().cwd().parent
DATA_PATH = REPO_BASEPATH / "pnet_data/raw/rho_small.npz"
CODE_PATH = REPO_BASEPATH / "src"
sys.path.append(str(CODE_PATH))
MODEL_CHECKPOINTS_PATH = REPO_BASEPATH / "results" / "models" / "pointnet_baseline.weights.h5"

import wandb
from data_viz import *
from model_utils import *

LABELS = ["unfocus hit", "focus hit"]

# set random seed for reproducibility
SEED = 18
set_global_seeds(SEED)

# data settings
N_TRAIN, N_VAL, N_TEST = 210, 65, 50 # roughly 0.65, 0.2, 0.15

# model settings
N_FEATURES = 3
INIT_SIZE = 8
END_SIZE = 16

# training settings
BATCH_SIZE = 16
EPOCHS = 20
INIT_LR = 0.003

2024-11-27 10:44:03.260757: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2024-11-27 10:44:05.239011: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT


## Model training

We proceed with model training:

1. split the data
1. build our PointNet model using Tensorflow/Keras
1. create a dataloader to feed batches into our model
1. train
1. check results

### PointNet model 

We use a PointNet model for semantic segmentation. Here is an illustration of its structure:

![PointNet architecture](../pnet_data/images/pointnet-architecture.jpg)

We have two heads:
 - classification head (used for point cloud classification)
 - segmentation head (used for semantic segmentation)

We are going to use the **segmentation head** for our problem. The architecture settings we can experiment with are:
 - `n_features` (the number of input features): original version has only size 3 as it only takes x,y,z coordinates
 - `init_size` (number of filters of first convolutional layer): original version has 64
 - `end_size` (number of filters in segmentation head): original version has 128

In [2]:
from wandb.integration.keras import WandbMetricsLogger, WandbModelCheckpoint

def read_data(split, bin_cutoff=0.5, n_classes=2):
    split_data_path = DATA_PATH.parent.parent
    filepath=str(split_data_path / f"{split}_data" / DATA_PATH.name)
    data = np.load(filepath)['feats']
    target_class = [(energy_fraction > bin_cutoff).astype(np.float32) 
                    for energy_fraction in data['truth_cell_focal_fraction_energy']]
    # target_class = (events["truth_cell_focal_fraction_energy"] > 0.5).reshape(-1)
    target_class = keras.utils.to_categorical(target_class, num_classes=n_classes)
    return data, target_class
    

def train_wrapper():
    run = wandb.init(project="mlops-ai_infn", entity="lclissa", # name="first-sweep", # not needed as the sweep takes care of this
                job_type="sweep", notes="Playing with sweeps ...")
    cfg = run.config
    input_features = ["normalized_x", "normalized_y", "normalized_z"]

    train_data, train_label_cloud = read_data("train")
    
    train_point_clouds = train_data[input_features]
    total_training_examples = len(train_point_clouds)
    
    val_data, val_label_cloud = read_data("val")
    val_point_clouds = val_data[input_features]
    
    print("Num train point clouds:", len(train_point_clouds))
    print("Num train point cloud labels:", len(train_label_cloud))
    print("Num val point clouds:", len(val_point_clouds))
    print("Num val point cloud labels:", len(val_label_cloud))
    
    n_points = train_point_clouds[0].shape[0]
    n_features = len(train_point_clouds[0].dtype.names)
    n_classes = len(LABELS)

    _ = run.use_artifact("train_data:latest")
    _ = run.use_artifact("val_data:latest")
    
    train_dataset = generate_dataset(train_point_clouds, train_label_cloud, 
                                 bs=cfg.batch_size, n_points=n_points, n_features=n_features, labels=LABELS)
    val_dataset = generate_dataset(val_point_clouds, val_label_cloud, is_training=False, 
                                   bs=cfg.batch_size, n_points=n_points, n_features=n_features, labels=LABELS)

    
    steps_per_epoch = total_training_examples // cfg.batch_size
    total_training_steps = steps_per_epoch * EPOCHS
    lr_schedule = keras.optimizers.schedules.ExponentialDecay(
        initial_learning_rate=cfg.init_lr,
        decay_steps=steps_per_epoch * 5,
        decay_rate=0.5,
        staircase=True,
    )

    segmentation_model = get_shape_segmentation_model(n_points, n_classes, n_features,
                                                      INIT_SIZE, END_SIZE)
    segmentation_model.compile(
        optimizer=keras.optimizers.Adam(learning_rate=lr_schedule),
        loss=keras.losses.BinaryCrossentropy(),
        metrics=["accuracy"],
        jit_compile=False
    )

    MODEL_CHECKPOINTS_PATH.parent.mkdir(exist_ok=True, parents=True)
    history = segmentation_model.fit(
        train_dataset,
        validation_data=val_dataset,
        epochs=EPOCHS,
        callbacks=[
            WandbMetricsLogger(log_freq=5),
            WandbModelCheckpoint(
                       MODEL_CHECKPOINTS_PATH, #.parent / "model-{epoch:02d}-{val_loss:.2f}.weights.h5",
                       monitor="val_loss",
                       save_best_only=True,
                       save_weights_only=True,
                   )
        ],
    )
    

In [3]:
# 2: Define the search space
sweep_configuration = {
    "method": "bayes",
    "metric": {"goal": "minimize", "name": "epoch/val_loss"},
    "parameters": {
        "init_lr": {'min': 1e-4, 'max': 1e-2,
                    'distribution': 'log_uniform' },
        "batch_size": {"values": [16, 32]},
        "init_size": {"values": [8, 16]},
    },
}

# 3: Start the sweep
sweep_id = wandb.sweep(sweep=sweep_configuration, project="mlops-ai_infn", entity="lclissa")

wandb.agent(sweep_id, function=train_wrapper, count=60)

Failed to detect the name of this notebook, you can set it manually with the WANDB_NOTEBOOK_NAME environment variable to enable code saving.
wandb: Using wandb-core as the SDK backend.  Please refer to https://wandb.me/wandb-core for more information.
wandb: WARNING Malformed sweep config detected! This may cause your sweep to behave in unexpected ways.
wandb: WARNING To avoid this, please fix the sweep config schema violations below:
wandb: WARNING   Violation 1. init_lr uses log_uniform, where min/max specify base-e exponents. Use log_uniform_values to specify limit values.


Create sweep with ID: t4eb9re6
Sweep URL: https://wandb.ai/lclissa/mlops-ai_infn/sweeps/t4eb9re6


wandb: Agent Starting Run: j902kbvx with config:
wandb: 	batch_size: 32
wandb: 	init_lr: 1.0021356802263224
wandb: 	init_size: 16
Failed to detect the name of this notebook, you can set it manually with the WANDB_NOTEBOOK_NAME environment variable to enable code saving.
wandb: Currently logged in as: lclissa. Use `wandb login --relogin` to force relogin
wandb: WARNING Ignored wandb.init() arg project when running a sweep.
wandb: WARNING Ignored wandb.init() arg entity when running a sweep.


Num train point clouds: 210
Num train point cloud labels: 210
Num val point clouds: 65
Num val point cloud labels: 65


2024-11-27 10:44:12.354830: I external/local_xla/xla/stream_executor/cuda/cuda_executor.cc:998] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-L355
2024-11-27 10:44:12.405864: I external/local_xla/xla/stream_executor/cuda/cuda_executor.cc:998] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-L355
2024-11-27 10:44:12.408295: I external/local_xla/xla/stream_executor/cuda/cuda_executor.cc:998] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-

...

Num train point clouds: 210
Num train point cloud labels: 210
Num val point clouds: 65
Num val point cloud labels: 65
Epoch 1/20
14/14 ━━━━━━━━━━━━━━━━━━━━ 8s 100ms/step - accuracy: 0.8523 - loss: 81385672.0000 - val_accuracy: 0.1266 - val_loss: inf
Epoch 2/20
14/14 ━━━━━━━━━━━━━━━━━━━━ 1s 50ms/step - accuracy: 0.8544 - loss: 28571632.0000 - val_accuracy: 0.8734 - val_loss: inf
Epoch 3/20
14/14 ━━━━━━━━━━━━━━━━━━━━ 1s 50ms/step - accuracy: 0.8725 - loss: 5325093.5000 - val_accuracy: 0.8734 - val_loss: inf
Epoch 4/20
14/14 ━━━━━━━━━━━━━━━━━━━━ 1s 61ms/step - accuracy: 0.8807 - loss: 1222546.7500 - val_accuracy: 0.8734 - val_loss: 61907187734912253853858081194115072.0000
Epoch 5/20
14/14 ━━━━━━━━━━━━━━━━━━━━ 1s 62ms/step - accuracy: 0.8790 - loss: 1259051.2500 - val_accuracy: 0.1266 - val_loss: 1298006746901565853907642080034816.0000
Epoch 6/20
14/14 ━━━━━━━━━━━━━━━━━━━━ 1s 61ms/step - accuracy: 0.8802 - loss: 85034064.0000 - val_accuracy: 0.8820 - val_loss: 55312744648094485846711735418

batch/accuracy,▁▆▄▅▅▆▆▆▆▆▆▅▆▆▆▆▆▅▆▆▆▃▆▆▅▇▇▆▆▆▇▆▆▆▆▆▆▆█▆
batch/batch_step,▁▁▁▁▁▂▂▂▂▂▃▃▃▃▃▄▄▄▄▄▅▅▅▅▅▆▆▆▆▆▇▇▇▇▇▇▇███
batch/learning_rate,███████████▄▄▄▄▄▄▄▄▂▂▂▂▂▂▂▂▂▂▁▁▁▁▁▁▁▁▁▁▁
batch/loss,▁▆█▄▃▁▁▁▁▁▁▄▇▆▂▂▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁
epoch/accuracy,▃▁██▆█▆▆████████████
epoch/epoch,▁▁▂▂▂▃▃▄▄▄▅▅▅▆▆▇▇▇██
epoch/learning_rate,████▄▄▄▄▄▂▂▂▂▁▁▁▁▁▁▁
epoch/loss,█▂▁▁▁▅▁▂▁▁▁▁▁▁▁▁▁▁▁▁
epoch/val_accuracy,▁███▁███████████████
epoch/val_loss,█▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁
batch/accuracy,0.8788


wandb: Agent Starting Run: 3qhyzy45 with config:
wandb: 	batch_size: 16
wandb: 	init_lr: 1.005756602844774
wandb: 	init_size: 16
Failed to detect the name of this notebook, you can set it manually with the WANDB_NOTEBOOK_NAME environment variable to enable code saving.
wandb: WARNING Ignored wandb.init() arg project when running a sweep.
wandb: WARNING Ignored wandb.init() arg entity when running a sweep.


Num train point clouds: 210
Num train point cloud labels: 210
Num val point clouds: 65
Num val point cloud labels: 65
Epoch 1/20
14/14 ━━━━━━━━━━━━━━━━━━━━ 8s 100ms/step - accuracy: 0.8400 - loss: 31855916.0000 - val_accuracy: 0.8734 - val_loss: inf
Epoch 2/20
14/14 ━━━━━━━━━━━━━━━━━━━━ 1s 50ms/step - accuracy: 0.8846 - loss: 47357248.0000 - val_accuracy: 0.8734 - val_loss: inf
Epoch 3/20
14/14 ━━━━━━━━━━━━━━━━━━━━ 1s 51ms/step - accuracy: 0.8827 - loss: 859850.0000 - val_accuracy: 0.1266 - val_loss: inf
Epoch 4/20
14/14 ━━━━━━━━━━━━━━━━━━━━ 1s 60ms/step - accuracy: 0.8875 - loss: 251454.8750 - val_accuracy: 0.8734 - val_loss: 212969478885975127609873923047424.0000
Epoch 5/20
14/14 ━━━━━━━━━━━━━━━━━━━━ 1s 59ms/step - accuracy: 0.8710 - loss: 67697.8906 - val_accuracy: 0.8734 - val_loss: 38166973439221043326422614016.0000
Epoch 6/20
14/14 ━━━━━━━━━━━━━━━━━━━━ 1s 61ms/step - accuracy: 0.8804 - loss: 46584.4688 - val_accuracy: 0.8734 - val_loss: 516825453979148595991937024.0000
Epoch 7/20

batch/accuracy,▁▅█▇▆▇█▇▆▇▇▇▇▇█▆▆▇▆▇▇▇▅▆▇▄▇█▇▅▇▆▇▇▆▇▇▇▇▆
batch/batch_step,▁▁▁▁▁▂▂▂▃▃▃▃▃▃▃▄▄▄▄▄▅▅▅▅▅▅▆▆▆▆▆▆▇▇▇▇████
batch/learning_rate,███████████▄▄▄▄▄▄▄▄▄▂▂▂▂▂▂▂▁▁▁▁▁▁▁▁▁▁▁▁▁
batch/loss,▃▃█▄▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁
epoch/accuracy,▁███▆█████▅█████████
epoch/epoch,▁▁▂▂▂▃▃▄▄▄▅▅▅▆▆▇▇▇██
epoch/learning_rate,████▄▄▄▄▄▂▂▂▂▁▁▁▁▁▁▁
epoch/loss,█▆▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁
epoch/val_accuracy,██▁████▁▁▁▁▁▁▁▂▂▃▄▆▇
epoch/val_loss,█▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁
batch/accuracy,0.87385


In addition to the convenience of having W&B coordinating the hyperparameter search, wandb UI also offers useful visualizations out of the box that help exploring the results.

In particular, we can easily inspect what hyperparameters combinations led to best performance through the **parameter importance** and **parallel coordinates** panels:


![](../pnet_data/images/wandb-sweep.png)


<div class="alert alert-block alert-info">

For more info check [wandb.sweeps](https://docs.wandb.ai/guides/sweeps/)
</div>

# Beyond Basic MLOps: Additional W&B Features

### CI/CD Integration (Continuous Integration/Deployment)
> **What**: Integration of W&B with CI/CD pipelines through GitHub Actions or similar tools  
> **Benefits**: 
> - Automated model testing and validation on each code change
> - Consistent quality checks across team development
> - Automated deployment of validated models
> - Early detection of performance regressions

### Reporting
> **What**: Interactive, shareable documents combining experiment results, visualizations, and narrative  
> **Benefits**:
> - Create reproducible ML research reports
> - Share results with stakeholders through dynamic dashboards
> - Compare multiple experiments in a single view
> - Document model development journey and decisions

### Collaborative Development
> **What**: Team-oriented features including shared projects, model registry, and access controls  
> **Benefits**:
> - Centralized model and experiment tracking
> - Standardized workflows across team members
> - Knowledge sharing and reproducibility
> - Easy handoff between team members

[Learn more about CI/CD Integration →](https://docs.wandb.ai/guides/artifacts/project-scoped-automations)  
[Explore W&B Reports →](https://docs.wandb.ai/guides/reports/)  
[Team Collaboration Guide →](https://docs.wandb.ai/guides/registry)

*Note: These features are particularly valuable as projects scale from individual experimentation to team-wide ML development.*